In [1]:
import requests
import pandas as pd
import funcoes_suporte as f

A API do surveymoney retorna tanto as respostas quanto as possíveis opções de resposta com identificações numéricas
Por isso, na parte de extrair os dado tem duas etapas, extrair as respostas, e extrair os detalhes sobre as perguntas
E depois cruzar os dois para poder quantificar e identificar o que foi respondido em cada questionário

Nessa parte são criadas as urls para acessar tanto os questionários preenchidos, quanto os detalhes do questionário

In [2]:
# ID do survey na API
Survey_ID = '172901009'

In [ ]:
# Endpoints das respostas e do detalhe dos surveys
endpoint_respostas = "/v3/surveys/%s/responses" % Survey_ID
endpoint_detalhes_survey = "/v3/surveys/%s/details" % Survey_ID

In [ ]:
# caminho final para request
url_respostas = "%s%s" % ("https://api.surveymonkey.net", endpoint_respostas)
url_detalhes_survey = "%s%s" % ("https://api.surveymonkey.net", endpoint_detalhes_survey)


Nessa parte é feito o request dos dados dos questionários preenchidos para se extrair as respostas de cada questionário
O ID numérico que representa cada resposta preenchida em cada pergunta de cada um dos questionários respondidos
é armazenado em "respostas_final" seguindo a estrutura:
    
    {id_questionário: {id_pergunta: id_resposta}}


In [4]:
# Contem o ID de todos os questionários respondidos
json_respostas = f.make_request(url_respostas)

# Armazena todos os IDs de todos os quetionários preenchidos
ids_survey = [survey['id'] for survey in json_respostas['data']]

# Armazena o caminho para acessar as respostas de todos os questionários preenchidos
urls_survey = [url_respostas + '/%s/details' % id_ for id_ in ids_survey]

In [6]:
# Aplica a função acima para armazenar as respostas de todos os questionários
respostas_final = {id_:f.get_resposta(url_) for id_, url_ in zip(ids_survey, urls_survey)}

In [8]:
print(respostas_final.keys())

dict_keys(['10751399944', '10777673860', '10777820410', '10782757738', '10788047634', '10792995373', '10793092507', '10793028733', '10793874678', '10793878604', '10793877701', '10795074178', '10795178252', '10795186249', '10795218812', '10795865086', '10796038275', '10910009949'])


In [13]:
respostas_final['10777820410']

{'261458088': '1771127817',
 '261458089': '1771145661',
 '261458092': '1771148217',
 '261458094': '1771162347',
 '261458102': '9',
 '261458105': '9',
 '261458106': '1771165261',
 '261458107': '1771169397',
 '261458108': '1771121972'}

Nessa parte é extraído dos dados de detalhe do survey a identificação numérica de cada possível resposta 
para cada pergunta.

Existem 3 tipos de perguntas: as single_choice (opção única) e matrix (escala) possuem um ID para cada possível
resposta, e esses ids são armazenados em dicionários seguindo a estrutura:

    {id_pergunta:[(id_opção, representação no formulário), (id_opção, representação no formulário)]}

No caso de perguntas abertas, apenas o ID da pergunta é armazenado já que a representação real da resposta é a própria resposta inputada por quem preencheu o questionário

In [14]:
# Contem a identificação numérica, pontuação e demais detalhes das perguntas do questionário separados por página
json_survey = f.make_request(url_detalhes_survey)

# Cria-se dois dicionários separados para armazenar a identificação das respostas de acordo com o ID da opção de resposta
# Isso é feito já que a API entrega as respostas do tipo "matrix" e "single_choice" do questionário como um ID numérico
matrix = {}
single_choice = {}

# Armazena o ID de perguntas abertas
pergunta_aberta = []

In [15]:
# Percorre o json_survey
# O respectivo ID de cada opção de perguntas fechadas fica em: json['pages'][i]['questions'][j]['answers']['choices']
for i in range(len(json_survey['pages'])):
    for j in range(len(json_survey['pages'][i]['questions'])):
        questao = json_survey['pages'][i]['questions'][j]
        if questao['family'] == 'single_choice':
            single_choice[questao['id']] = questao['answers']['choices']
        elif questao['family'] == 'matrix':
            matrix[questao['id']] = questao['answers']['choices']
        else:
            pergunta_aberta.append(questao['id'])

In [16]:
# Funções para gerar tuplas com o Id e o segundo campo alvo para cada opção de cada pergunta
# No caso de perguntas do tipo matrix > (Id, Posição)
tupla_matrix = lambda dict_: (dict_['id'], dict_['position'])

# No caso de perguntas do tipo single_choise > (Id, Texto)
tupla_sc = lambda dict_: (dict_['id'], dict_['text'])

# Gera o dicionário contendo {id_pergunta:[(id_opção, representação real), (id_opção, representação real)]}
questoes_matrix = {id_questao:[tupla_matrix(dict_) for dict_ in matrix[id_questao]] for id_questao in matrix}
questoes_single_choice = {id_questao:[tupla_sc(dict_) for dict_ in single_choice[id_questao]] for id_questao in single_choice}
questoes_abertas = {id_questao: None for id_questao in pergunta_aberta}

# Dicionário final, contendo todas as questões e suas respectivas identificações por opção
dict_final = {**questoes_matrix,
              **questoes_single_choice,
              **questoes_abertas}

In [20]:
f.gerar_dataframe(respostas_final, dict_final).head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
10751399944,5,4,5,5,Sim,Sim,Não,Sim,Sim,3 ou mais,...,None,None,None,None,16 a 30,6,6,Juliana,Ana Lídia Cunha,4632
10777673860,4,None,None,None,Não,Sim,Não,None,None,None,...,None,None,None,None,None,None,None,None,None,None
10777820410,4,5,None,None,Não,Sim,Não,Não se aplica,Sim,None,...,None,None,None,None,None,9,9,None,None,None
10782757738,5,5,5,5,Sim,Sim,Não,Sim,Sim,3 ou mais,...,Sim,None,None,None,1 a 15,4,10,Juliana,Caio Montagner,3710
10788047634,None,None,None,None,None,None,None,None,None,None,...,Sim,None,None,None,None,None,None,None,None,None
